Projeto: Análise de atrito de funcionários RH IBM.
Descrição: Projeto de ciência de dados para prever a saída (attrition) de funcionários, 
utilizando o dataset IBM HR Analytics. O objetivo principal é apoiar a área de 
Recursos Humanos (RH) na implementação de estratégias de retenção proativas.

Autor: Ivan Luís Duarte

LinkedIn: https://www.linkedin.com/in/ivanluisduarte/

GitHub: https://github.com/ivanluisduarte

Data de Criação: 2025-10-23

Licença: MIT

# Preparando os componentes da aplicação

A aplicação precisa conter as opções válidas para o modelo. Vamos aqui preparar os ranges e opções válidas para cada coluna que se tornará entrada na aplicação.

## Importações

In [1]:
from src.config import DCT_CONFIGURACAO_COLUNAS, DADOS_LIMPOS


import pandas as pd
import re

from json import dump
from pprint import pprint

## Função auxiliar
Separa as palavras que compõe o nome da coluna

In [2]:
# Função para separar camel case
def fnc_separar_camel_case(texto_camel_case: str) -> str:
    '''
        Separa uma string em camel case, inserindo espaços entre as palavras.

        Args:
            texto_camel_case (str): A string em camel case a ser separada.  Exemplo: "YearsWithCurrManager".
        Returns:
            str: A string separada com espaços entre as palavras. Exemplo: "Years With Curr Manager".
    '''

    # A expressão regular (r"(\w)([A-Z])") procura:
    # 1. (\w): qualquer caractere de palavra (letra, número ou underscore) - grupo 1
    # 2. ([A-Z]): uma letra maiúscula - grupo 2
    # E substitui por:
    # r"\1 \2": o caractere do grupo 1, um espaço, e o caractere do grupo 2.
    texto_camel_case = re.sub(r"(\w)([A-Z])", r"\1 \2", texto_camel_case)

    # 2. Tratar o Hífen (substitui "-" por " ")
    texto_camel_case = texto_camel_case.replace("-", " - ")

    return texto_camel_case

## Carregando dados

In [3]:
lst_colunas_modelo = [
    'BusinessTravel', 'DailyRate', 'Department-JobRole',
    'EnvironmentSatisfaction', 'JobSatisfaction', 'MaritalStatus',
    'NumCompaniesWorked', 'OverTime', 'StockOptionLevel', 'TotalWorkingYears',
    'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole',
    'YearsSinceLastPromotion', 'YearsWithCurrManager',
]

In [4]:
df = pd.read_parquet(path=DADOS_LIMPOS, columns=lst_colunas_modelo)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1470 entries, 1 to 2068
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   BusinessTravel           1470 non-null   category
 1   DailyRate                1470 non-null   uint16  
 2   Department-JobRole       1470 non-null   category
 3   EnvironmentSatisfaction  1470 non-null   category
 4   JobSatisfaction          1470 non-null   category
 5   MaritalStatus            1470 non-null   category
 6   NumCompaniesWorked       1470 non-null   uint8   
 7   OverTime                 1470 non-null   category
 8   StockOptionLevel         1470 non-null   category
 9   TotalWorkingYears        1470 non-null   uint8   
 10  WorkLifeBalance          1470 non-null   category
 11  YearsAtCompany           1470 non-null   uint8   
 12  YearsInCurrentRole       1470 non-null   uint8   
 13  YearsSinceLastPromotion  1470 non-null   uint8   
 14  YearsWithCurr

In [5]:
df.head()

,BusinessTravel,DailyRate,Department-JobRole,EnvironmentSatisfaction,JobSatisfaction,MaritalStatus,NumCompaniesWorked,OverTime,StockOptionLevel,TotalWorkingYears,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1,Travel_Rarely,1102,Sales - Sales Executive,Medium,Very High,Single,8,Yes,0,8,Bad,6,4,0,5
2,Travel_Frequently,279,Research & Development - Research Scientist,High,Medium,Married,1,No,1,10,Better,10,7,1,7
4,Travel_Rarely,1373,Research & Development - Laboratory Technician,Very High,High,Single,6,Yes,0,7,Better,0,0,0,0
5,Travel_Frequently,1392,Research & Development - Research Scientist,Very High,High,Married,1,Yes,0,8,Better,8,7,3,0
7,Travel_Rarely,591,Research & Development - Laboratory Technician,Low,Medium,Married,9,No,1,6,Better,2,2,2,2


## Separando entre numéricas e categóricas

In [6]:
lst_colunas_categoricas = df.select_dtypes(exclude='number').columns.tolist()
lst_colunas_numericas = df.select_dtypes(include='number').columns.tolist()

print(f'Colunas Categóricas: {lst_colunas_categoricas}')
print(f'Colunas Numéricas: {lst_colunas_numericas}')

Colunas Categóricas: ['BusinessTravel', 'Department-JobRole', 'EnvironmentSatisfaction', 'JobSatisfaction', 'MaritalStatus', 'OverTime', 'StockOptionLevel', 'WorkLifeBalance']
Colunas Numéricas: ['DailyRate', 'NumCompaniesWorked', 'TotalWorkingYears', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']


## Preparando um dicionário para os valores

Sem isso o python se perde usando a mesma referência de dicionário.

In [7]:
# Inicializa uma estrutura de base (que será copiada)
estrutura_base_categoricas = {'label': None, 'mode': None, 'categories': []}
estrutura_base_numericas = {'label': None,
                            'mode': None, 'min': None, 'max': None}

# Inicializa o dicionário com uma cópia independente para cada coluna
dct_configuracao_colunas = {
    'colunas_categoricas': {coluna: estrutura_base_categoricas.copy()
                            for coluna in lst_colunas_categoricas},
    'colunas_numericas':  {coluna: estrutura_base_numericas.copy()
                           for coluna in lst_colunas_numericas},
}

pprint(dct_configuracao_colunas)

{'colunas_categoricas': {'BusinessTravel': {'categories': [],
                                            'label': None,
                                            'mode': None},
                         'Department-JobRole': {'categories': [],
                                                'label': None,
                                                'mode': None},
                         'EnvironmentSatisfaction': {'categories': [],
                                                     'label': None,
                                                     'mode': None},
                         'JobSatisfaction': {'categories': [],
                                             'label': None,
                                             'mode': None},
                         'MaritalStatus': {'categories': [],
                                           'label': None,
                                           'mode': None},
                         'OverTime': {'categories': [],
     

## Estabelecendo opções das categóricas

In [8]:
for coluna in lst_colunas_categoricas:
    dct_configuracao_colunas['colunas_categoricas'][coluna]['label'] = fnc_separar_camel_case(
        coluna)
    dct_configuracao_colunas['colunas_categoricas'][coluna]['mode'] = df[coluna].mode()[
        0]
    dct_configuracao_colunas['colunas_categoricas'][coluna]['categories'] = df[coluna].cat.categories.tolist()

pprint(dct_configuracao_colunas)

{'colunas_categoricas': {'BusinessTravel': {'categories': ['Non-Travel',
                                                           'Travel_Frequently',
                                                           'Travel_Rarely'],
                                            'label': 'Business Travel',
                                            'mode': 'Travel_Rarely'},
                         'Department-JobRole': {'categories': ['Human '
                                                               'Resources - '
                                                               'Human '
                                                               'Resources',
                                                               'Human '
                                                               'Resources - '
                                                               'Manager',
                                                               'Research & '
                             

## Estabelecendo os limites das numéricas

In [9]:
for coluna in lst_colunas_numericas:
    dct_configuracao_colunas['colunas_numericas'][coluna]['label'] = fnc_separar_camel_case(
        coluna)
    dct_configuracao_colunas['colunas_numericas'][coluna]['mode'] = int(
        df[coluna].mode())
    dct_configuracao_colunas['colunas_numericas'][coluna]['min'] = int(
        df[coluna].min())
    dct_configuracao_colunas['colunas_numericas'][coluna]['max'] = int(
        df[coluna].max())

pprint(dct_configuracao_colunas)

{'colunas_categoricas': {'BusinessTravel': {'categories': ['Non-Travel',
                                                           'Travel_Frequently',
                                                           'Travel_Rarely'],
                                            'label': 'Business Travel',
                                            'mode': 'Travel_Rarely'},
                         'Department-JobRole': {'categories': ['Human '
                                                               'Resources - '
                                                               'Human '
                                                               'Resources',
                                                               'Human '
                                                               'Resources - '
                                                               'Manager',
                                                               'Research & '
                             

## Salvando o dicionário

In [10]:
with open(DCT_CONFIGURACAO_COLUNAS, 'w') as f:
    dump(obj=dct_configuracao_colunas, fp=f)